In [1]:
%matplotlib widget
import os
import numpy as np
import pandas as pd
import textwrap
from astropy.io import fits
import scipy.constants as const
import single_lc_funcs as fn
import matplotlib.pyplot as plt

In [2]:
tex_fonts = {
    # Use LaTeX to write all text
    "text.usetex": True,
    "font.family": "serif",
    # Use 10pt font in plots, to match 10pt font in document
    "axes.labelsize": 13,
    "font.size": 12,
    # Make the legend/label fonts a little smaller
    "legend.fontsize": 13,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
}

plt.rcParams.update(tex_fonts)

In [3]:
#Load in occurence rate tables
fgk = pd.read_csv(os.path.join('Occurrence_rates','norm_rates_FGK.csv'))
tables = {'F': fgk, 'G':fgk, 'K':fgk}

In [4]:
#Filter target list to include only single sector targets
target_list_path = os.path.join('target-lists', 'year1targets.csv')
target_list = pd.read_csv(target_list_path, index_col=None)
target_list['N_sec'] = target_list.drop('OBJECT', axis=1).sum(axis=1)
mask = target_list['N_sec'] == 1
target_list = target_list[mask]
del target_list['N_sec']

In [5]:
#Pick a target
lim = target_list.shape[0]
tic, secs = fn.tic_choose(target_list, np.random.randint(low=0, high=lim))
Paths = fn.spoc_lc_path(tic, secs)

In [6]:
#Generate transit signals and calculate signal to noise
obj = fn.signals(path=Paths[0], number=100, rate_tables=tables, N_b=10, N_phase=10)
obj.get_results()
log = obj.to_pandas()
#print(log)
log.to_csv('sample_output1.csv')
st_str = ('Radius='+str(round(obj.st_rad,2))+', Type='+obj.spectral_type+', TMag='+str(round(obj.mag, 2)))
print(st_str)

Radius=0.94, Type=G, TMag=11.6


In [7]:
#Plot sector lightcurve
lc_str = (st_str+', SEC:'+str(obj.lc.sector)+', CAM:'+str(obj.lc.cam)+', CCD:'+str(obj.lc.ccd)+', noise='+str(round(obj.lc.noise2hr,2))+', variability='+str(round(obj.lc.var,2)))
fig, ax = plt.subplots(1,1,figsize=[12.8,4.8], num=0)
ax.errorbar(x=obj.lc.time, y=obj.lc.flux, yerr=obj.lc.flux_err)
ax.set_xlabel('Time (TBJD)')
ax.set_ylabel('PDCSAP FLUX $(e^{-}s^{-1})$')
ax.set_title(lc_str)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#Plot signal to noise as a function of period and radius
S_N = log['S/N']
rad = log['PL_RADIUS']
period = log['PL_PERIOD']
fig, ax = plt.subplots(1,2, figsize=[12.8, 4.8], num=1)
ax[0].scatter(x=rad, y=S_N)
ax[0].set_xlabel('Radius (Earth Radii)')
ax[0].set_ylabel('Signal to Noise ratio')
ax[1].scatter(x=period, y =S_N)
ax[1].set_xlabel('Orbital Period (days)')
ax[1].set_ylabel('Signal to noise ratio')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#Plot histogram of phases, T_0 and b
fig, ax = plt.subplots(1,3, figsize=(6.4*3, 4.8), num=2)
phase_off = np.ndarray.flatten(obj.ph_offset)
T0 = np.ndarray.flatten(obj.T_0)
ax[0].hist(x=phase_off, bins=20)
ax[0].set_xlabel('Phase offset')
ax[0].set_xlim(.0, 1.)
ax[1].hist(x=T0, bins=20)
ax[1].set_xlabel('$T_0$')
ax[2].hist(x=np.ndarray.flatten(obj.b), bins=20)
ax[2].set_xlabel('Impact Parameter, b')
ax[2].set_xlim(.0, 1.)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
#Plot phase offset against Signal to noise: colour coded for b:
N_pl = len(obj.pl_period)
fig, ax = plt.subplots(1, int(N_pl), figsize=[6.4*N_pl, 4.8], num=3)
for i in range(N_pl):
    for j in range(len(obj.b[i])):
        b_str = 'b=' + str(round(obj.b[i,j], 3))
        ax[i].scatter(x=obj.ph_offset[i,j], y=obj.SN[i,j], label=b_str)
        ax[i].set_xlabel('Phase offset')
        ax[i].set_ylabel('Signal to noise ratio')
        title_str = ('Radius ='+str(round(obj.pl_rad[i], 2))+' Period='+str(round(obj.pl_period[i],2)))
        ax[i].set_title(title_str)
        ax[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()


In [ ]:
#Plot T_0 against Signal to noise, colour coded for b
N_pl = len(obj.pl_period)
fig, ax = plt.subplots(1, int(N_pl), figsize=[6.4*N_pl, 4.8], num=4)
for i in range(N_pl):
    for j in range(len(obj.b[i])):
        b_str = 'b=' + str(round(obj.b[i,j], 3))
        ax[i].scatter(x=obj.T_0[i,j], y=obj.SN[i,j], label=b_str)
        ax[i].set_xlabel('$T_0$ (TBJD)')
        ax[i].set_ylabel('Signal to noise ratio')
        title_str = ('Radius ='+str(round(obj.pl_rad[i], 2))+' Period='+str(round(obj.pl_period[i],2)))
        ax[i].set_title(title_str)
        ax[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))
fig.tight_layout()
plt.show()

In [ ]:
#Plot impact parameter vs S/N averaged across all phases
N_pl = len(obj.pl_period)
fig, ax = plt.subplots(1, int(N_pl), figsize=[6.4*N_pl, 4.8], num=5)
for i in range(N_pl):
    S_N = np.empty_like(obj.b[i])
    for j in range(len(obj.b[i])):
        S_N[j] = np.mean(obj.SN[i,j])
    ax[i].scatter(x=obj.b[i], y=S_N)
    ax[i].set_xlabel('Impact parameter')
    ax[i].set_ylabel('Mean S/N')
    title_str = ('Radius ='+str(round(obj.pl_rad[i], 2))+' Period='+str(round(obj.pl_period[i],2)))
    ax[i].set_title(title_str)
fig.tight_layout()
plt.show()

In [ ]:
#Choose a signal by row in the pd dataframe and visually inject it into the lc
sig_ch = 0 #Input integer index of row
fig, ax = plt.subplots(2,1, figsize=[12.8, 9.6], num=6)